In [ ]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam

# Load ResNet50 model with ImageNet weights, excluding the top layers
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze the ResNet layers to retain pre-trained knowledge
for layer in resnet.layers:
    layer.trainable = False

# Initialize a sequential model
model = Sequential()

# Add the ResNet50 base
model.add(resnet)

# Add a GlobalAveragePooling layer to reduce dimensions before the dense layers
model.add(GlobalAveragePooling2D())

# Add fully connected layers for the specific classification task
model.add(Dense(64, activation='relu'))  # First Dense Layer
model.add(Dropout(0.25))  # Dropout to prevent overfitting

model.add(Dense(30, activation='relu'))  # Intermediate Dense Layer
model.add(Dropout(0.25))  # Dropout to prevent overfitting

# Final output layer for 15 classes (sports personnel classification)
model.add(Dense(15, activation='softmax'))

# Compile the model with an optimizer and loss function for multiclass classification
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Model summary to check architecture
model.summary()


In [ ]:
# Optionally unfreeze the deeper layers for fine-tuning after initial training
for layer in resnet.layers[-10:]:  # Unfreeze last 10 layers
    layer.trainable = True

# Recompile the model after unfreezing layers
model.compile(optimizer=Adam(learning_rate=0.00001),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Rescale pixel values from 0-255 to 0-1
    shear_range=0.2,         # Random shear transformations
    zoom_range=0.2,          # Random zoom transformations
    horizontal_flip=True,    # Random horizontal flips
    validation_split=0.2     # Reserve 20% of data for validation
)

# ImageDataGenerator for validation, without augmentation, just rescaling
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training images from a directory with subdirectories representing classes
train_generator = train_datagen.flow_from_directory(
    "C:/Users/vamsh/Downloads/datathon-24/train/train",  
    target_size=(64, 64),     # Resize images to 64x64 pixels
    batch_size=32,            # Number of images to yield per batch
    class_mode='categorical', # Multiclass classification
    subset='training'         # Specify this as the training set
)

# Load validation images from the same directory
val_generator = val_datagen.flow_from_directory(
    "C:/Users/vamsh/Downloads/datathon-24/train/train",  # Replace with your training data path
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'       # Specify this as the validation set
)


Found 373 images belonging to 15 classes.
Found 87 images belonging to 15 classes.


In [7]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,                    # Adjust number of epochs as needed
    validation_data=val_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)


Epoch 1/20


C:\Users\vamsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.0534 - loss: 2.8860 - val_accuracy: 0.0575 - val_loss: 2.7462
Epoch 2/20


C:\Users\vamsh\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 635ms/step - accuracy: 0.0693 - loss: 2.8316 - val_accuracy: 0.0575 - val_loss: 2.7427
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 607ms/step - accuracy: 0.0499 - loss: 2.8093 - val_accuracy: 0.0575 - val_loss: 2.7381
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 526ms/step - accuracy: 0.0742 - loss: 2.7720 - val_accuracy: 0.0460 - val_loss: 2.7323
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 578ms/step - accuracy: 0.0663 - loss: 2.7586 - val_accuracy: 0.0460 - val_loss: 2.7271
Epoch 10/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/20
12/12 ━━━━━━━━━━━━━━━━━━━

# Testing the data

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Path to the folder containing the test images
test_image_folder = "C:/Users/vamsh/Downloads/datathon-24/test/test"

# Use the same preprocessing as your training set
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a list to store predictions and true labels
predictions = []

# Loop through each image in the test folder
for img_name in os.listdir(test_image_folder):
    img_path = os.path.join(test_image_folder, img_name)
    
    # Load the image and resize to the same size as the model input
    img = image.load_img(img_path, target_size=(64, 64))
    
    # Convert the image to an array and expand dimensions to match the input shape
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess the image (rescaling, etc.)
    img_array = test_datagen.standardize(img_array)  # Rescale as in training
    
    # Predict the class
    pred = model.predict(img_array)
    predictions.append(pred.argmax(axis=1)[0])

# Convert predictions to numpy array for further analysis
predictions = np.array(predictions)

# If you have ground truth labels, compare them for accuracy
# Example: assuming `true_labels` is a list of ground truth labels
# true_labels = [...]

# You can calculate accuracy manually like this (if you have true labels)
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(true_labels, predictions)
# print(f"Accuracy: {accuracy * 100:.2f}%")



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms

In [20]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Path to the folder containing the test images
test_image_folder = "C:/Users/vamsh/Downloads/datathon-24/test/test"

# Use the same preprocessing as your training set
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a list to store predictions
predictions = []

# Load and predict a few images
for img_name in os.listdir(test_image_folder)[:10]:  # Adjust the number of images
    img_path = os.path.join(test_image_folder, img_name)
    
    # Load the image and preprocess
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = test_datagen.standardize(img_array)
    
    # Predict the class
    pred = model.predict(img_array)
    predicted_class = pred.argmax(axis=1)[0]
    
    # Print the prediction
    print(f"Image: {img_name}, Predicted class: {predicted_class}")
    predictions.append(predicted_class)


# Use the same preprocessing as your training set
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a list to store predictions
predictions = []

# Load and predict a few images
for img_name in os.listdir(test_image_folder)[:10]:  # Adjust the number of images
    img_path = os.path.join(test_image_folder, img_name)
    
    # Load the image and preprocess
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = test_datagen.standardize(img_array)
    
    # Predict the class
    pred = model.predict(img_array)
    predicted_class = pred.argmax(axis=1)[0]
    
    # Print the prediction
    print(f"Image: {img_name}, Predicted class: {predicted_class}")
    predictions.append(predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Image: 01d55934f2.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Image: 0203170f6a.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Image: 021f017430.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Image: 045c0e444e.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Image: 05aba9afc8.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Image: 078411b7f3.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Image: 088d9410e5.jpg, Predicted class: 10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Image: 09b7a14201.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Image: 0acb08b02f.jpg, Predicted class: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Image: 0cee7c1751.jpg, Predicted class: 10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Image: 01d55934f2.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Image: 0203170f6a.jpg, Predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

# Predicting single image

In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to a single image for prediction
img_path = "C:/Users/vamsh/Downloads/datathon-24/test/test/e4c9668613.jpg"

# Load the image with the same target size as the training images (64x64)
img = image.load_img(img_path, target_size=(64, 64))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Reshape the image for the model (add batch dimension)
img_array = np.expand_dims(img_array, axis=0)

# Normalize the pixel values (as done during training)
img_array /= 255.0

# Make the prediction
predictions = model.predict(img_array)

# Get the class with the highest predicted probability
predicted_class = np.argmax(predictions, axis=1)

print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Predicted class: [10]


In [6]:
train_set = train_gen.flow_from_directory(, target_size=(64,64), class_mode='categorical', batch_size=32)

Found 460 images belonging to 15 classes.


In [7]:
print(train_set.class_indices)

{'bhuvneshwar_kumar': 0, 'dinesh_karthik': 1, 'hardik_pandya': 2, 'jasprit_bumrah': 3, 'k._l._rahul': 4, 'kedar_jadhav': 5, 'kuldeep_yadav': 6, 'mohammed_shami': 7, 'ms_dhoni': 8, 'ravindra_jadeja': 9, 'rohit_sharma': 10, 'shikhar_dhawan': 11, 'vijay_shankar': 12, 'virat_kohli': 13, 'yuzvendra_chahal': 14}


In [8]:
test_set = test_gen.flow_from_directory("C:/Users/vamsh/Downloads/datathon-24/test", target_size=(64,64), class_mode='categorical', batch_size=32)

Found 116 images belonging to 1 classes.


In [9]:
model.fit(train_set, epochs = 25)

Epoch 1/25


C:\Users\vamsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 196ms/step - accuracy: 0.0717 - loss: 2.8538
Epoch 2/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 188ms/step - accuracy: 0.0786 - loss: 2.6991
Epoch 3/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.0789 - loss: 2.6976
Epoch 4/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 179ms/step - accuracy: 0.1048 - loss: 2.6764
Epoch 5/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 190ms/step - accuracy: 0.1256 - loss: 2.6373
Epoch 6/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.1095 - loss: 2.6265
Epoch 7/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 181ms/step - accuracy: 0.1100 - loss: 2.6097
Epoch 8/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 189ms/step - accuracy: 0.1437 - loss: 2.5710
Epoch 9/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.1755 - loss: 2.5161
Epoch 10/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.2392 - loss: 2.4642
Epoch 11/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - accuracy: 0.2054 - loss: 2.4716
Epoch 12/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step - accuracy

In [10]:
ypred = model.predict(test_set)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step


In [11]:
ypred

array([[9.6187443e-02, 5.0531290e-03, 1.6516628e-02, ..., 2.8449628e-03,
        7.8438902e-03, 4.7466378e-03],
       [9.5315516e-02, 4.8545562e-02, 3.3233482e-02, ..., 1.6571507e-01,
        5.3190678e-02, 2.4162540e-02],
       [2.2143383e-02, 5.3586280e-03, 2.6653159e-02, ..., 3.3511585e-04,
        1.6360225e-02, 7.6918341e-02],
       ...,
       [1.7529565e-05, 5.4993373e-03, 8.9971902e-05, ..., 2.5299423e-05,
        8.9547676e-01, 4.8305136e-03],
       [2.9975340e-02, 2.3794359e-02, 4.9582735e-02, ..., 1.0905992e-02,
        1.2768412e-02, 2.9912766e-02],
       [2.6048064e-02, 7.4461782e-03, 2.3948614e-02, ..., 3.8640136e-03,
        1.7354863e-02, 7.9666413e-02]], dtype=float32)